In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import spacy

In [4]:
from nltk import WordNetLemmatizer, PorterStemmer

In [13]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


False

In [10]:
#from google.colab import drive
#drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [14]:
df = pd.read_csv("Dataset.csv", index_col='tweet_id')

FileNotFoundError: [Errno 2] No such file or directory: 'Dataset.csv'

In [ ]:
df = df.iloc[:, [9,0]]

In [ ]:
df

In [ ]:
df_train = df[: int(0.8*df.shape[0])]

In [ ]:
df_test = df[int(0.8*df.shape[0]):]

In [ ]:
print(f"Total Neutral Count - {df[df['airline_sentiment']=='neutral'].shape[0]}")
print(f"Total Positive Count - {df[df['airline_sentiment']=='positive'].shape[0]}")
print(f"Total Negative Count - {df[df['airline_sentiment']=='negative'].shape[0]}")

In [ ]:
print(f"Train Neutral Count - {df_train[df_train['airline_sentiment']=='neutral'].shape[0]}")
print(f"Train Positive Count - {df_train[df_train['airline_sentiment']=='positive'].shape[0]}")
print(f"Train Negative Count - {df_train[df_train['airline_sentiment']=='negative'].shape[0]}")

In [ ]:
print(f"Test Neutral Count - {df_test[df_test['airline_sentiment']=='neutral'].shape[0]}")
print(f"Test Positive Count - {df_test[df_test['airline_sentiment']=='positive'].shape[0]}")
print(f"Test Negative Count - {df_test[df_test['airline_sentiment']=='negative'].shape[0]}")

#Text cleaning and character embedding

In [ ]:
import re

In [ ]:
nlp = spacy.load('en_core_web_sm')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    # translator = str.maketrans('', '', string.punctuation)
    # text = text.translate(translator)

    # Remove stop words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    useless_words = useless_words + ['hi', 'im']

    text_filtered = [word for word in text if not word in useless_words]

    # # Remove numbers
    # text_filtered = [re.sub(r'\w*\d\w*', '', w) for w in text_filtered]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string

In [ ]:
df = df_train

In [ ]:
df['text'] = df['text'].apply(lambda x: clean_string(x))
df_test['text'] = df_test['text'].apply(lambda x: clean_string(x))

In [ ]:
df

##padding with space

In [ ]:
lens = df['text'].apply(lambda x : len(x))
np.max(lens)

In [ ]:
def pad_string(st):
    z = 160 - len(st)
    st = st + (' '*z)
    return st

In [ ]:
df['text'] = df['text'].apply(lambda x: pad_string(x))
df_test['text'] = df_test['text'].apply(lambda x: pad_string(x))

In [ ]:
# df.to_csv("/content/drive/MyDrive/kaggle/DL_assignment/train_unaug.csv")

In [ ]:
# df_test.to_csv("/content/drive/MyDrive/kaggle/DL_assignment/test_unaug.csv")

In [ ]:
# df = pd.read_csv("/content/drive/MyDrive/kaggle/DL_assignment/train_unaug.csv", index_col='tweet_id')

In [ ]:
# df_test = pd.read_csv("/content/drive/MyDrive/kaggle/DL_assignment/test_unaug.csv", index_col='tweet_id')

##Embedding

In [ ]:
import string
def string_vectorizer(strng, alphabet=string.ascii_lowercase + string.digits + string.punctuation):
    vector = [[0 if char != letter else 1 for char in alphabet] 
                  for letter in strng]
    return vector

In [ ]:
embedded_text = []
for t in df['text']:
    v = string_vectorizer(t)
    embedded_text.append(v)
embedded_text = np.array(embedded_text)

In [ ]:
embedded_test = []
for t in df_test['text']:
    v = string_vectorizer(t)
    embedded_test.append(v)
embedded_test = np.array(embedded_test)

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

In [ ]:
# ohc = OneHotEncoder(handle_unknown='ignore')

In [ ]:
# to_fit = list(set(''.join(list(df['text']))))

In [ ]:
# to_fit = np.array(to_fit).reshape(-1,1)

In [ ]:
# to_test = []
# for s in df_test['text']:
#     to_test.append(list(s))

In [ ]:
# ohc  = ohc.fit(to_fit)

In [ ]:
# ohc.transform([['a']]).toarray().flatten()

In [ ]:
# train_feat = []
# for s in df['text']:
#     train_feat.append([ohc.transform([[i]]).toarray().flatten() for i in list(s)])

In [ ]:
# test_feat = []
# for s in df_test['text']:
#     test_feat.append([ohc.transform([[i]]).toarray().flatten() for i in list(s)])

In [ ]:
# train_feat = np.array(train_feat)
# test_feat = np.array(test_feat)

In [ ]:
embedded_text=embedded_text.astype('float64')
embedded_test=embedded_test.astype('float64')

In [ ]:
train_feat = embedded_text
test_feat = embedded_test

#Encode labels

In [ ]:
pd.get_dummies(df['airline_sentiment'])

In [ ]:
train_labels = np.array(pd.get_dummies(df['airline_sentiment']))
test_labels = df_test['airline_sentiment']

In [ ]:
val_labels = np.array(pd.get_dummies(df_test['airline_sentiment']))

In [ ]:
# train_labels = one_hot_labels[:int(0.8*embedded_text.shape[0])]
# test_labels = df['airline_sentiment'][int(0.8*embedded_text.shape[0]):]

In [ ]:
# test_label = one_hot_labels[int(0.8*embedded_text.shape[0]):]

In [ ]:
train_labels

#model

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv1D(64, 3, activation='relu'),
        tf.keras.layers.MaxPool1D(strides=2),
        tf.keras.layers.Conv1D(128, 3, activation='relu'),
        tf.keras.layers.MaxPool1D(strides=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ]
)

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics='accuracy')

In [ ]:
model.fit(train_feat, train_labels, epochs=4, validation_data=(test_feat, val_labels))

In [ ]:
pred = model.predict(test_feat)

In [ ]:
model.summary()

In [ ]:
model.save('/content/drive/MyDrive/DL_Assignment/final_model')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
prediction=[]
for p in pred:
    xx = np.argmax(p)
    if xx==2:
        prediction.append('positive')
    elif xx==1:
        prediction.append('neutral')
    else:
        prediction.append('negative')

In [ ]:
print("Accuracy: ", accuracy_score(test_labels, prediction))
print("Macroaveraged precision: ", precision_score(test_labels, prediction, average='macro'))
print("Macroaveraged recall: ", recall_score(test_labels, prediction, average='macro'))
print("Macro f1: ", f1_score(test_labels, prediction, average='macro'))